# Example of using either HEAAN or SEAL


HEAAN과 MS-SEAL은 모두 CKKS scheme을 지원한다. 그러나 implementation간 기능의 차이가 존재하며, 라이브러리의 구조와 지향하는 바도 조금씩 다르다. 
따라서 완전한 통합은 어렵고, 기본적인 기능을 동일한 interface로 활용하도록 하는데 목표를 둔다. 

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from fase.core import common
import numpy as np

from fase import seal
from fase import HEAAN as he

from fase.core.common import HEAANContext, HEAANParameters, SEALContext

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### SEAL과 HEAN의 CKKS Context를 만든 뒤, 두 context에 대해 동일한 interface로 작업을 수행 

* SEAL은 BFV와 CKKS를 공통적으로 지원하는 라이브러이기이 때문에 SEAL과 HEAAN의 파라미터 구성에 상당한 차이가 있다.
* 파라미터 구성은 어플리케이션 마다 사용자가 직접 지정해주어야한다. 

In [7]:
# SEAL setup
poly_modulus_degree = 16384
coeff_moduli = [37, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 37]

ckks_se = SEALContext(poly_modulus_degree=poly_modulus_degree,
                             coeff_moduli=coeff_moduli,
                             scale_bit=28)
# HEAAN setup
ckks_he = HEAANContext(2, 30, 150)

SEAL CKKS scheme is ready
saving secret key done.
HEAAN CKKS setup is ready 


## encrypt and decrypt

In [8]:
for ckks in [ckks_he, ckks_se]:
    val = [1,2,3,4]
    ctxt = ckks.encrypt(val)
    dec = ckks.decrypt(ctxt)
    print(dec[:len(val)])
    print(ckks._name)

[1.00000003 2.00000002 2.99999998 3.99999997]
HEAAN
[1.00002571 2.0000022  3.00000298 4.00000428]
SEAL


## Add

* ckks supports direct addition and subtraction between Ciphertext and Plaintext

In [9]:
for ckks in [ckks_he, ckks_se]:
    v1 = [1,2,3,4]
    v2 = [5,6,7,8]
    ctxt1 = ckks.encrypt(v1)
    ctxt2 = ckks.encrypt(v2)
    ctxt3 = ckks.add(ctxt1, ctxt2)
    print(ckks.decrypt(ctxt3)[:len(v1)])
    
    # inplace option
    ckks.add(ctxt1, ctxt2, inplace=True)
    print(ckks.decrypt(ctxt1)[:len(v1)])
    
    print(ckks._name)

[ 6.          7.99999998  9.99999999 11.99999994]
[ 6.          7.99999998  9.99999999 11.99999994]
HEAAN
[ 5.99997846  8.00000222 10.0000105  11.99998789]
[ 5.99997846  8.00000222 10.0000105  11.99998789]
SEAL


# Subtract 

In [10]:
for ckks in [ckks_he, ckks_se]:
    v1 = [1,2,3,4]
    v2 = [5,6,7,8]
    ctxt1 = ckks.encrypt(v1)
    ctxt2 = ckks.encrypt(v2)
    ctxt3 = ckks.sub(ctxt1, ctxt2)
    print(ckks.decrypt(ctxt3)[:len(v1)])
    
    # inplace option
    ckks.sub(ctxt1, ctxt2, inplace=True)
    print(ckks.decrypt(ctxt1)[:len(v1)])
    
    print(ckks._name)

[-4.         -3.99999995 -4.00000011 -4.00000019]
[0.99999995 2.00000009 2.99999995 3.99999989]
HEAAN
[-3.99999977 -3.99999598 -4.00000245 -3.99999218]
[-3.99999977 -3.99999598 -4.00000245 -3.99999218]
SEAL


## Multiply by const value

CKKS scheme은 *plain number* **-(encode)->** *plain text* **-(encrypt)>** *cipher text* 순서로 변환이 진행된다.  
Plaintext는 plain number와 Ciphertext의 중간 단계로 생각할 수 있다.  

SEAL은 Ciphertext와 plaintext 사이의 계산도 지원함.  
HEAAN은 plaintext 단계를 implicit하게 지나감.  

In [11]:
for ckks in [ckks_he, ckks_se]:
    v1 = [1,2,3,4]
    v2 = [5,6,7,8]
    ctxt1 = ckks.encrypt(v1)
    ctxt2 = ckks.multByConst(ctxt1, v2)
    print(ckks.decrypt(ctxt2)[:len(v1)])
    
    ckks.multByConst(ctxt1, v2, inplace=True)
    print(ckks.decrypt(ctxt1)[:len(v1)], "inplace=True")
    print(ckks._name)

[ 5.00000024 10.00000026 15.00000025 20.00000009]
[ 5.00000024 10.00000026 15.00000025 20.00000009] inplace=True
HEAAN
[ 5.00007507 11.99998222 20.99999925 32.00002732]
[ 5.00007507 11.99998222 20.99999925 32.00002732] inplace=True
SEAL


## Multiply by Ciphertext

In [12]:
for ckks in [ckks_he, ckks_se]:
    v1 = [1,2,3,4]
    v2 = [5,6,7,8]
    ctxt1 = ckks.encrypt(v1)
    ctxt2 = ckks.encrypt(v2)
    ctxt3 = ckks.mult(ctxt1, ctxt2, inplace=False)
    print(ckks.decrypt(ctxt3)[:len(v1)])
    
    # inplace option
    ckks.mult(ctxt1, ctxt2, inplace=True)
    print(ckks.decrypt(ctxt1)[:len(v1)])
    
    print(ckks._name)

[ 4.99999932 12.00000052 21.0000002  31.99999971]
[ 4.99999932 12.00000052 21.0000002  31.99999971]
HEAAN


TypeError: relin() takes 2 positional arguments but 3 were given

## Square

In [ ]:
for ckks in [ckks_he, ckks_se]:
    v1 = [1,2,3,4]
    ctxt1 = ckks.encrypt(v1)
    ctxt3 = ckks.square(ctxt1)
    print(ckks.decrypt(ctxt3)[:len(v1)])
    
    #inplace option
    ckks.square(ctxt1, inplace=True)
    print(ckks.decrypt(ctxt1)[:len(v1)])
    
    print(ckks._name)

## Rescale
HEAAN과 SEAL은 rescaling 방식이 조금 다르다.  

SEAL은 momulus switch로 coeff_moduli에서 설정해준 다음 modulus로 넘어가는 방식이고, 
HEAAN은 자동적으로 ctxt.logp 만큼의 scale을 줄인다.  
이에 따라 HEAAN은 ctxt.logq가 noise budget이 되고, SEAL에서는 coeff_moduli의 개수로 multiplication depth가 정해진다. 

* rescale()은 inplace=True를 기본으로 가정한다 

In [13]:
# Different approach to checking scale 
for ckks in [ckks_he, ckks_se]:
    v1 = [1,2,3,4]
    ctxt1 = ckks.encrypt(v1)

    if ckks._name == "HEAAN":
        print("initial scale", ctxt1.logp)
        ctxt3 = ckks.square(ctxt1)

        print("scale after suqare()", ctxt3.logp)
        ckks.rescale(ctxt3)
        print("scale after rescaling()", ctxt3.logp)
        print('\n')
        
    elif ckks._name == "SEAL":
        print("Initial Scale", np.log2(ctxt1.scale()))
        ckks.square(ctxt1, inplace=True)
        ckks._evaluator.relinearize_inplace(ctxt1, ckks_se.relin_keys) # relinearization
        print("Scale after square", np.log2(ctxt1.scale()))

        # Rescale
        ckks.rescale(ctxt1)
        print("Scale after rescaling", np.log2(ctxt1.scale()))
        
        print("after manual fix", np.log2(ctxt1.scale()))
        print('\n')

initial scale 30
scale after suqare() 60
scale after rescaling() 30


Initial Scale 28.0
Scale after square 56.0
Scale after rescaling 28.0
after manual fix 28.0




## Mod switch

Rescaling은 Ciphertext의 scale을 줄이는 대신 noise budget을 감소시켜 rescaling을 겪지 않은 Ciphertext와 추가적인 계산을 불가능하게 만든다.  
이를 해결하려면 새 Ciphertext의 mod를 다른 Ciphertext의 mod에 맞춰 변경해줘야한다.  

SEAL에서는 modulus chain의 index가 같은지 확인해야하고, HEAAN은 ctxt.logq가 같은지 확인 해야함. 

In [14]:
# Different approach to checking scale 
for ckks in [ckks_he, ckks_se]:
    v1 = [1,2,3,4]
    ctxt1 = ckks.encrypt(v1)
    new_ct = ckks.encrypt([3,4,5,6])

    if ckks._name == "HEAAN":
        print("initial scale and mod", ctxt1.logp, ctxt1.logq)
        ctxt1 = ckks.square(ctxt1)

        print("scale after suqare()", ctxt1.logp)
        ckks.rescale(ctxt1)
        print("scale and mod after rescaling()", ctxt1.logp, ctxt1.logq)
        print("new_ct's mod", new_ct.logq)
        
        ckks.match_mod(new_ct, ctxt1)
        print("new_ct's mod switched", new_ct.logq)
        
    elif ckks._name == "SEAL":
        print("Initial Scale", np.log2(ctxt1.scale()))
        ckks.square(ctxt1, inplace=True)
        ckks._evaluator.relinearize_inplace(ctxt1, ckks_se.relin_keys) # relinearization
        print("Scale after square", np.log2(ctxt1.scale()))

        # Rescale
        ckks.rescale(ctxt1)
        print("Scale after rescaling", np.log2(ctxt1.scale()))
        
        print("after manual fix", np.log2(ctxt1.scale()))
        print("modulus after rescaling", ckks.context.get_context_data(ctxt1.parms_id()).chain_index())
        
        print("new_ct's modulus index", ckks.context.get_context_data(new_ct.parms_id()).chain_index())
        
        ckks.match_mod(new_ct, ctxt1)
        print("new_ct's modulus index after mod switch", ckks.context.get_context_data(new_ct.parms_id()).chain_index())
        print('\n')
    
    
    ckks.add(new_ct, ctxt1, inplace=True)
    print(ckks.decrypt(new_ct)[:len(v1)])
    print("Correct result = [4,8,14,22]")
    print('\n')


initial scale and mod 30 150
scale after suqare() 60
scale and mod after rescaling() 30 120
new_ct's mod 150
new_ct's mod switched 120
[ 3.99999999  8.0000002  13.99999997 21.99999996]
Correct result = [4,8,14,22]


Initial Scale 28.0
Scale after square 56.0
Scale after rescaling 28.0
after manual fix 28.0
modulus after rescaling 12
new_ct's modulus index 13
new_ct's modulus index after mod switch 12


[ 4.00029153  8.00096052 14.00221702 22.00385786]
Correct result = [4,8,14,22]


